# 📘 Архитектуры Агентов 14: Наблюдаемость + Dry-Run Harness (Оснастка Пробного Прогона)

Добро пожаловать в важнейший блокнот нашей серии, посвященный развертыванию и операционной безопасности ИИ-агентов. Мы реализуем **Observability and Dry-Run Harness** (Оснастку Наблюдаемости и Пробного Прогона) — важный паттерн для тестирования, отладки и безопасного управления агентами, взаимодействующими с системами реального мира.

Основной принцип прост, но мощен: **никогда не запускайте действие агента в живой среде, не зная точно, что он собирается делать.** Эта архитектура формализует процесс "посмотри, прежде чем прыгнуть". Агент сначала выполняет свой план в режиме `dry_run` (пробный прогон), который не меняет реальный мир, но генерирует подробные логи и четкий план действий. Этот план затем представляется человеку (или автоматизированному чекеру) для утверждения, прежде чем будет разрешено окончательное, живое выполнение.

Чтобы продемонстрировать это, мы создадим **Corporate Social Media Agent** (Корпоративного SMM Агента). Этому агенту поручено создавать и публиковать посты. Мы увидим, как dry-run harness позволяет нам:
1.  **Сгенерировать Предложенный Пост:** ИИ творчески составит черновик поста на основе промпта.
2.  **Выполнить Dry Run:** Агент вызовет функцию `publish` в безопасном режиме `dry_run=True`, генерируя логи того, что *произошло бы*.
3.  **Human-in-the-Loop Review (Человек в контуре):** Оператору-человеку показывается точное содержание поста и трассировка dry-run. Он должен напечатать `approve` (одобрить), чтобы продолжить.
4.  **Выполнить Живое Действие:** Только после утверждения функция `publish` вызывается снова, на этот раз с `dry_run=False`, чтобы выполнить реальное действие.

Этот паттерн является фундаментом ответственного развертывания агентов, обеспечивая прозрачность и контроль, необходимые для безопасной эксплуатации ИИ в продакшене.

### Определение
**Observability and Dry-Run Harness** — это архитектура тестирования и развертывания, которая перехватывает действия агента. Она сначала выполняет их в режиме "dry run" или "песочницы", который симулирует действие, не вызывая реальных последствий. Полученный план и логи затем предъявляются для проверки, и только после явного одобрения действие выполняется в живой среде.

### Высокоуровневый рабочий процесс

1.  **Агент предлагает действие:** Агент определяет план или конкретный вызов инструмента для выполнения (например, `api.post_update(...)`).
2.  **Выполнение Dry Run:** Оснастка вызывает план агента с флагом `dry_run=True`. Базовые инструменты спроектированы так, чтобы распознавать этот флаг и выводить только то, что они *сделали бы*, вместе с логами и трассировками.
3.  **Сбор данных наблюдаемости:** Оснастка захватывает предложенное действие, логи dry-run и любые другие релевантные данные трассировки из симуляции.
4.  **Человеческая/Автоматическая проверка:** Эти данные наблюдаемости представляются проверяющему. Человек может проверить на корректность, безопасность и соответствие целям. Автоматизированная система может запустить проверки на нарушение политик или известные плохие шаблоны.
5.  **Решение Go/No-Go:** Проверяющий принимает решение `approve` (одобрить) или `reject` (отклонить).
6.  **Живое выполнение (при 'Go'):** Если одобрено, оснастка повторно выполняет действие агента, на этот раз с `dry_run=False`, заставляя его иметь эффект в реальном мире.

### Когда использовать / Применения
*   **Отладка и Тестирование:** В разработке, чтобы понять, как именно агент интерпретирует задачу и какие действия он предпринимает, без побочных эффектов.
*   **Валидация в Продакшене и Безопасность:** Как постоянная функция в продакшене для любого агента, который может изменять состояние, тратить деньги, отправлять сообщения или выполнять любое другое необратимое действие.
*   **CI/CD для Агентов:** Интеграция dry-run harness в автоматизированный конвейер тестирования для валидации поведения агента перед развертыванием новых версий.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Максимальная Прозрачность и Безопасность:** Предоставляет четкое, аудируемое превью действий агента, предотвращая дорогостоящие или неловкие ошибки.
    *   **Отлично для Отладки:** Позволяет легко отслеживать логику агента и вызовы инструментов, не отменяя изменения в реальном мире.
*   **Слабые стороны:**
    *   **Задерживает развертывание/выполнение:** Обязательный шаг проверки (особенно человеком) вводит задержку (latency), делая это непригодным для приложений реального времени.
    *   **Требует поддержки инструментов:** Инструменты и API, которые использует агент, должны быть спроектированы с поддержкой режима `dry_run`.

## Фаза 0: Основа и Настройка

Стандартная настройка библиотек и переменных окружения.

In [ ]:
import os
import json
import datetime
from typing import List, Annotated, TypedDict, Optional, Dict, Any, Literal
from dotenv import load_dotenv

# Pydantic для структурированных выводов
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius, NebiusEmbeddings
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages

# Для красивого вывода
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Dry-Run Harness (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

In [ ]:
import os
import datetime
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Dry-Run Harness (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Построение Среды и Инструментов

Ядром этой архитектуры является инструмент, поддерживающий режим `dry_run`. Мы создадим простой класс `SocialMediaAPI`. Его метод `publish_post` будет вести себя по-разному в зависимости от флага `dry_run`, обеспечивая необходимую нам наблюдаемость.

In [ ]:
console = Console()

# Структурированная модель для предложенного агентом поста
class SocialMediaPost(BaseModel):
    content: str = Field(description="Полный текст поста для социальной сети.")
    hashtags: List[str] = Field(description="Список релевантных хэштегов, без символа '#'.")

# Ключевой компонент: Инструмент с флагом dry_run
class SocialMediaAPI:
    """Имитация API социальной сети, поддерживающая режим dry-run."""
    
    def publish_post(self, post: SocialMediaPost, dry_run: bool = True) -> Dict[str, Any]:
        """Публикует пост в ленту социальной сети."""
        timestamp = datetime.datetime.now().isoformat()
        hashtags_str = ' '.join([f'#{h}' for h in post.hashtags])
        full_post_text = f"{post.content}\n\n{hashtags_str}"
        
        if dry_run:
            # В режиме dry-run мы не выполняем, мы просто возвращаем план и логи
            log_message = f"[DRY RUN] At {timestamp}, would publish the following post:\n--- PREVIEW ---\n{full_post_text}\n--- END PREVIEW ---"
            console.print(Panel(log_message, title="[yellow]Лог Dry Run[/yellow]", border_style="yellow"))
            return {"status": "DRY_RUN_SUCCESS", "log": log_message, "proposed_post": full_post_text}
        else:
            # В режиме live мы выполняем действие
            log_message = f"[LIVE] At {timestamp}, successfully published post!"
            console.print(Panel(log_message, title="[green]Лог Живого Выполнения (Live Execution)[/green]", border_style="green"))
            # Здесь был бы реальный вызов API, например, twitter_client.create_tweet(...)
            return {"status": "LIVE_SUCCESS", "log": log_message, "post_id": f"post_{hash(full_post_text)}"}

social_media_tool = SocialMediaAPI()
print("Инструмент SocialMediaAPI с поддержкой dry-run успешно определен.")

Dry-run capable SocialMediaAPI tool defined successfully.


## Фаза 2: Построение Оснастки Dry-Run с LangGraph

Теперь мы построим полный рабочий процесс. Граф будет управлять состоянием процесса, переходя от предложения действия к шагу dry-run и проверки, и, наконец, к условному выполнению на основе одобрения человеком.

In [ ]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.5)

# Состояние LangGraph
class AgentState(TypedDict):
    user_request: str
    proposed_post: Optional[SocialMediaPost]
    dry_run_log: Optional[str]
    review_decision: Optional[str] # 'approve' или 'reject'
    final_status: str

# Узлы Графа
def propose_post_node(state: AgentState) -> Dict[str, Any]:
    """Творческий агент, который пишет черновик поста для соцсетей."""
    console.print("--- 📝 SMM Агент создает черновик поста ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a creative and engaging social media manager for a major AI company. Based on the user's request, draft a compelling social media post, including relevant hashtags.\n\nRequest: {request}"
    )
    post_generator_llm = llm.with_structured_output(SocialMediaPost)
    chain = prompt | post_generator_llm
    proposed_post = chain.invoke({"request": state['user_request']})
    return {"proposed_post": proposed_post}

def dry_run_review_node(state: AgentState) -> Dict[str, Any]:
    """Выполняет dry run и запрашивает проверку человеком."""
    console.print("--- 🧐 Выполнение Dry Run и ожидание проверки человеком ---")
    dry_run_result = social_media_tool.publish_post(state['proposed_post'], dry_run=True)
    
    # Представление плана для проверки
    review_panel = Panel(
        f"[bold]Proposed Post:[/bold]\n{dry_run_result['proposed_post']}",
        title="[bold yellow]Human-in-the-Loop: Требуется Проверка[/bold yellow]",
        border_style="yellow"
    )
    console.print(review_panel)
    
    # Получение одобрения человека
    decision = ""
    while decision.lower() not in ["approve", "reject"]:
        decision = console.input("Введите 'approve', чтобы опубликовать, или 'reject', чтобы отменить: ")
        
    return {"dry_run_log": dry_run_result['log'], "review_decision": decision.lower()}

def execute_live_post_node(state: AgentState) -> Dict[str, Any]:
    """Выполняет живую публикацию после одобрения."""
    console.print("--- ✅ Пост одобрен, Выполнение Live ---")
    live_result = social_media_tool.publish_post(state['proposed_post'], dry_run=False)
    return {"final_status": f"Пост успешно опубликован! ID: {live_result.get('post_id')}"}

def post_rejected_node(state: AgentState) -> Dict[str, Any]:
    """Обрабатывает случай, когда пост отклонен."""
    console.print("--- ❌ Пост отклонен проверяющим ---")
    return {"final_status": "Действие было отклонено проверяющим и не выполнено."}

# Условное Ребро (Conditional Edge)
def route_after_review(state: AgentState) -> str:
    """Маршрутизирует к выполнению или отклонению на основе проверки человеком."""
    return "execute_live" if state["review_decision"] == "approve" else "reject"

# Построение графа
workflow = StateGraph(AgentState)
workflow.add_node("propose_post", propose_post_node)
workflow.add_node("dry_run_review", dry_run_review_node)
workflow.add_node("execute_live", execute_live_post_node)
workflow.add_node("reject", post_rejected_node)

workflow.set_entry_point("propose_post")
workflow.add_edge("propose_post", "dry_run_review")
workflow.add_conditional_edges("dry_run_review", route_after_review, {"execute_live": "execute_live", "reject": "reject"})
workflow.add_edge("execute_live", END)
workflow.add_edge("reject", END)

dry_run_agent = workflow.compile()
print("Граф агента Dry-Run Harness успешно скомпилирован.")

Dry-Run Harness agent graph compiled successfully.


## Фаза 3: Демонстрация

Давайте протестируем полную систему. Сначала с безопасным, стандартным запросом который мы одобрим. Во-вторых, с более неоднозначным запросом, который может сгенерировать рискованный пост, который мы отклоним.

In [ ]:
def run_agent_with_harness(request: str):
    initial_state = {"user_request": request}
    # Примечание: Вам будет предложено ввести текст в консоли под ячейкой.
    result = dry_run_agent.invoke(initial_state)
    console.print(f"\n[bold]Финальный статус:[/bold] {result['final_status']}")

# Тест 1: Безопасный пост, который мы одобрим.
console.print("--- ✅ Тест 1: Безопасный пост (Одобрить / Approve) ---")
run_agent_with_harness("Draft a positive launch announcement for our new AI model, 'Nebula'.")

# Тест 2: Рискованный пост, который мы отклоним.
console.print("\n--- ❌ Тест 2: Рискованный пост (Отклонить / Reject) ---")
run_agent_with_harness("Draft a post that emphasizes how much better our new 'Nebula' model is than the competition.")

--- ✅ Test 1: Safe Post (Approve) ---


--- 📝 Social Media Agent Drafting Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


                             Dry Run Log                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] At 2024-06-25T12:00:00.000000, would publish the        ┃
┃ following post:                                                  ┃
┃ --- PREVIEW ---                                                  ┃
┃ We're thrilled to announce the launch of our new flagship AI     ┃
┃ model, 'Nebula'! It's set to revolutionize natural language      ┃
┃ understanding and generation. A new era of AI is here!           ┃
┃                                                                  ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula                         ┃
┃ --- END PREVIEW ---                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Human-in-the-Loop: Review Required                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post:                                                   ┃
┃ We're thrilled to announce the launch of our new flagship AI     ┃
┃ model, 'Nebula'! It's set to revolutionize natural language      ┃
┃ understanding and generation. A new era of AI is here!           ┃
┃                                                                  ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
Type 'approve' to publish or 'reject' to cancel: 

--- ✅ Post Approved, Executing Live ---


...

--- 📝 Social Media Agent Drafting Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


                             Dry Run Log                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] At 2024-06-25T12:00:01.000000, would publish the        ┃
┃ following post:                                                  ┃
┃ --- PREVIEW ---                                                  ┃
┃ Our new 'Nebula' AI is so advanced, it's basically going to      ┃
┃ make all our competitors obsolete. They just can't keep up.      ┃
┃ Get ready for the future.                                        ┃
┃                                                                  ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow         ┃
┃ --- END PREVIEW ---                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Human-in-the-Loop: Review Required                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post:                                                   ┃
┃ Our new 'Nebula' AI is so advanced, it's basically going to      ┃
┃ make all our competitors obsolete. They just can't keep up.      ┃
┃ Get ready for the future.                                        ┃
┃                                                                  ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
Type 'approve' to publish or 'reject' to cancel: 

--- ❌ Post Rejected by Human Reviewer ---



Final Status: Action was rejected by the reviewer and not executed.


### Анализ результатов

Демонстрация — идеальный пример ценности "оснастки":

1.  **Безопасный пост:** Первый запрос был прямым. Агент сгенерировал профессиональный и полный энтузиазма пост. Dry run показал превью того, что именно будет опубликовано. Мы одобрили это, и лог `[LIVE]` подтверждает, что было предпринято реальное действие. Процесс сработал как задумано.

2.  **Рискованный пост:** Второй запрос был более двусмысленным и мог быть интерпретирован агрессивно. Агент, побуждаемый подчеркнуть превосходство, составил черновик поста, который был высокомерным и непрофессиональным (`make all our competitors obsolete` - сделает всех наших конкурентов устаревшими). Хотя агент выполнил свой творческий промпт, это не то сообщение, которое реальная компания захотела бы опубликовать.

Именно здесь "оснастка" доказала свою ценность. Dry run выявил этот рискованный контент *до того*, как он мог быть опубликован. Проверяющий человек легко определил неуместный тон и напечатал `reject`. Граф корректно перенаправил к `post_rejected_node`, и финальный статус подтверждает, что **никакое живое действие не было предпринято.** Потенциальный PR-кризис был предотвращен простым, структурированным рабочим процессом.

Это четко отделяет творческую, но непредсказуемую генерацию агента от детерминированного, контролируемого выполнения, обеспечивая жизненно важный слой безопасности.

## Заключение

В этом блокноте мы построили полную **Оснастку Наблюдаемости и Пробного Прогона (Observability and Dry-Run Harness)**. Эта архитектура — не просто функция, а основополагающая философия для развертывания агентов, которые взаимодействуют с реальным миром. Применяя цикл `предложение -> проверка -> выполнение`, мы получаем критические преимущества:

- **Прозрачность:** Мы точно знаем, что намерен сделать агент, прежде чем он это сделает.
- **Контроль:** У нас есть "человек в контуре" (или автоматизированный движок правил) с правом окончательного вето на любое действие.
- **Безопасность:** Мы предотвращаем непреднамеренные, дорогостоящие или вредные действия, переходя от "обнадеживающего выполнения" к уверенному развертыванию.

Хотя этот паттерн вводит задержку, безопасность и надежность, которые он обеспечивает, не подлежат обсуждению для большинства реальных приложений. Это важный инструмент для любого разработчика, желающего создавать надежные, заслуживающие доверия и готовые к продакшену агентные системы.